In [0]:
!pip install pydantic-ai
!pip install "pydantic-ai-slim[openai]"

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
%restart_python

In [0]:

ACTION_SEND_EMAIL = "ACTION_SEND_EMAIL"
ACTION_NOTHING = "ACTION_NOTHING"
ACTION_CALL_CUSTOMER="ACTION_CALL_CUSTOMER",
CTION_ESCALATE_TICKET="ACTION_ESCALATE_TICKET",
CLOSE_TICKET = "CLOSE_TICKET"
FIND_NEW_TECH = "FIND_NEW_TECH"
ESCALATE_TO_SCOPE_EXPERT = "ESCALATE_TO_SCOPE_EXPERT"
ESCALATE_TO_BILLING_TEAM = "ESCALATE_TO_BILLING_TEAM"
CREATE_CHANGE_ORDER = "CREATE_CHANGE_ORDER"
GET_UPDATED_ETA = "GET_UPDATED_ETA"



In [0]:
class AbstractDataConnector:
    def __init__(self):
        pass

    def get_email_activity(self):
        pass

class DatabaseDataConnector(AbstractDataConnector):
    def __init__(self):
        pass

    def get_email_activity(self):
        return {
            "subject": "Drive way paving",
            "content": "We have a driveway that needs to be paved. Can I get someone out"
        }

class MockDataConnector(AbstractDataConnector):
    def __init__(self, mock_data: dict):
        self.mock_data = mock_data

    def get_email_activity(self):
        return self.mock_data["get_email_activity"]

db_connector = DatabaseDataConnector()

class FeedbackData:
    def __init__(self):
        self.feedback = []

    def get_feedback(self):
        return self.feedback
    
    def add_feedback(self, fb):
        self.feedback.append(fb)

import nest_asyncio
nest_asyncio.apply()


from pydantic_ai import Agent
from pydantic_ai import Agent, ModelRetry, RunContext

import os

from dataclasses import dataclass

os.environ["OPENAI_API_KEY"] = dbutils.secrets.get("openai", "api-key")

SYSTEM_PROMPT = """
You are an event driven assistant that recommends actions to parterns.
You work for TBS (TELS Building Services).
Customers email you with service information.

Please call tools to gather context and then call one of the action tools to generate an aciton/

Examples (Event -> Action): 
Customer email with questions
When is tech going to arrive -> reach out to tech for updated ETA
Issue with project scope, want to see change -> escalate to scoping expert + communicate to SP
Tech did not finish issue -> *decision* follow up with tech or find a new tech
ETA is not satisfactory -> find new tech
Bill does not match quote -> escalate to SP + billing team
Other -> do nothing
Service Provider/Technician emails with issues 
Change order needed -> create change order and send to customer for approval
Tech Delayed -> inform customer + update system
Other -> do nothing

"""

class CustomerServiceAgent:
    pass

@dataclass
class AgentContext:
    agent: CustomerServiceAgent
    feedback_source: FeedbackData

class CustomerServiceAgent:
    def __init__(self, connector):
        self.actions_called = []
        self.data_connector = connector

        pydanitc_agent = Agent(
            'openai:gpt-4o',
            system_prompt=SYSTEM_PROMPT
        )

        ########## EXTERNAL DATA SOURCE TOOLS ##########

        @pydanitc_agent.tool
        async def get_email_activity(
            ctx: RunContext[AgentContext]
        ) -> dict[str, float]:
            """Gets the activity of an email
            Args:
                ctx: The context.
            """
            print(f"  TOOL CALL: get_email_activity")
            return connector.get_email_activity()
        
        @pydanitc_agent.tool
        async def get_feedback(
            ctx: RunContext[AgentContext]
        ):
            """Gets useful rules and prior feedback data to help make a decision
            Args:
                ctx: The context.
            """
            # As this grows, we can use RAG
            feedback = ctx.deps.feedback_source.get_feedback()
            if len(feedback) == 0:
                return "No feedback provided as of yet"
            feedback_str = "\n".join(feedback)

            return feedback_str


        ########## ACTION TOOLS ##########

        @pydanitc_agent.tool
        async def action_send_email(
            ctx: RunContext[AgentContext], event_identified:str,
            email_purpose: str
        ) -> dict[str, float]:
            """Recommends an email action
            Args:
                ctx: The context.
                event_identified: The event this fell under.
                email_purpose: The purpose of what the email should be about. This should be useful to a sales partner
            """
            print(f"  TOOL CALL: action_send_email: {email_purpose}")
            ctx.deps.agent.actions_called.append({
                "action": ACTION_SEND_EMAIL,
                "event_identified": event_identified,
                "metadata": {
                    "reason": email_purpose,
                    "params": {"address":"TODO@gmail.com", "subject": "TODO", "body": "TODO"}
                }
            })
            return {'status': 'success'}

        @pydanitc_agent.tool
        async def action_get_updated_eta(
            ctx: RunContext[AgentContext], event_identified:str,
            reason: str
        ) -> dict[str, float]:
            """Contact the tech and get an updated ETA
            Args:
                ctx: The context.
                event_identified: The event this fell under.
                reason: The reason why an updated ETA is needed
            """
            print(f"  TOOL CALL: get_ETA: {reason}")
            ctx.deps.agent.actions_called.append({
                "action": GET_UPDATED_ETA,
                "event_identified": event_identified,
                "metadata": {
                    "reason": reason,
                    "params": {"ticketNumber":"123asd"}
                }
            })
            return {'status': 'success'}
        
        # ACTION_CALL_CUSTOMER
        @pydanitc_agent.tool
        async def action_call_customer(
            ctx: RunContext[AgentContext], event_identified:str,
            call_purpose: str
        ) -> dict[str, float]:
            """Recommends an call action
            Args:
                ctx: The context.
                event_identified: The event this fell under.
                call_purpose: The purpose of what the call should be about. This should be useful to a sales partner
            """
            print(f"  TOOL CALL: action_call_customer: {call_purpose}")
            ctx.deps.agent.actions_called.append({
                "action": "ACTION_CALL_CUSTOMER",
                "event_identified": event_identified,
                "metadata": {
                    "reason": call_purpose,
                    "params": {"number":"123-456-7890"}
                }
            })
            return {'status': 'success'}

        # ACTION_ESCALATE_TICKET
        @pydanitc_agent.tool
        async def action_escalate_ticket(
            ctx: RunContext[AgentContext], event_identified:str,
            escalation_reason: str
        ) -> dict[str, float]:
            """Recommends an ticket escalation
            Args:
                ctx: The context.
                event_identified: The event this fell under.
                escalation_reason: The reason for escalation of the ticket. This should be useful to a sales partner
            """
            print(f"  TOOL CALL: action_escalate_ticket: {escalation_reason}")
            ctx.deps.agent.actions_called.append({
                "action": "ACTION_ESCALATE_TICKET",
                "event_identified": event_identified,
                "metadata": {
                    "reason": escalation_reason
                }
            })
            return {'status': 'success'}
        
        @pydanitc_agent.tool
        async def action_close_ticket(
            ctx: RunContext[AgentContext], event_identified:str, 
            reason: str
        ) -> dict[str, float]:
            """Recommends agent closes ticket
            Args:
                ctx: The context.
                event_identified: The event this fell under. 
                reason: The reason for closure
            """
            print(f"  TOOL CALL: action_close_ticket: {reason}")
            ctx.deps.agent.actions_called.append({
                "action": CLOSE_TICKET,
                "event_identified": event_identified,
                "metadata": {
                    "reason": reason
                }
            })
            return {'status': 'success'}

        @pydanitc_agent.tool
        async def action_fallback_to_customer_service(
            ctx: RunContext[AgentContext], event_identified:str, reason: str
        ) -> dict[str, float]:
            """If we cannot find a proper action for the agent to take, then we fallback to customer service. 
            Args:
                ctx: The context.
                event_identified: The event this fell under. This should always be UNSUPPORTED_EVENT 
                reason: The reason this communication does not fall under another event and needs to be looked at manually.
            """
            ctx.deps.agent.actions_called.append({
                "action": ACTION_NOTHING,
                "event_identified": event_identified,
                "metadata": {
                    "reason": reason
                }
            })
            return {'status': 'success'}
        
        self.agent = pydanitc_agent
            
    def invoke_agent(self, message, feedback_source):
        self.actions_called = []
        self.agent.run_sync(
            message,
            deps=AgentContext(agent=self, feedback_source=feedback_source)
        )
        actions_called = self.actions_called
        data_state = {
            "get_email_activity": self.data_connector.get_email_activity()
        }
        return {
            "actions": actions_called,
            "data_state": data_state
        }


default_feedback_source = FeedbackData()

cs_agent = CustomerServiceAgent(db_connector)
res = cs_agent.invoke_agent("New email came in for customer. Please gather context and recommend an action.", default_feedback_source)
print(res["actions"])

  TOOL CALL: get_email_activity
  TOOL CALL: action_send_email: Gather more details about the driveway paving project and discuss scheduling a site visit or service appointment.
[{'action': 'ACTION_SEND_EMAIL', 'event_identified': 'Customer Inquiry for Service', 'metadata': {'reason': 'Gather more details about the driveway paving project and discuss scheduling a site visit or service appointment.', 'params': {'address': 'TODO@gmail.com', 'subject': 'TODO', 'body': 'TODO'}}}]


In [0]:
%sql
Select * from hackathon.default.event_log

EventID,EventType,EventFrom,SenderType,Subject,Content,Summary,ActionSuggested,insrt_tmstmp,insrt_ID
1,Email,pqr@abc.com,Technician,Updated service quote available,"As requested, please find the updated service quote. Let us know if it fits your requirements.",Technician provided updated service quote.,CREATE_CHANGE_ORDER,2025-05-23,System
2,Call,9876543210,Technician,Updated ETA for service,We’ve updated the estimated time of arrival for your scheduled service visit. Thank you for your patience.,Technician provided updated ETA.,GET_UPDATED_ETA,2025-06-06,System
3,SalesPitch,System,System,"New Senior Living Facility Launch in Columbus, OH","We're excited to inform you about a new senior living facility under construction in Columbus, OH. Since you've purchased a AC unit from us, we think you'll love our new range of air purifiers that complement your existing choice. Take advantage of our early access deals!","New senior facility in Columbus, OH; recommended air purifier for existing AC unit buyer.",SALES CALL,2025-06-01,System
4,SalesPitch,System,System,"New Senior Living Facility Launch in Orlando, FL","We're excited to inform you about a new senior living facility under construction in Orlando, FL. Since you've purchased a AC unit from us, we think you'll love our new range of air purifiers that complement your existing choice. Take advantage of our early access deals!","New senior facility in Orlando, FL; recommended air purifier for existing AC unit buyer.",SALES CALL,2025-05-15,System
5,Email,abc@xyz.com,Customer,Billing discrepancy,I noticed an unexpected charge on my bill this month. Can someone explain?,Customer inquired about a billing issue.,ESCALATE_TO_BILLING_TEAM,2025-05-18,System
6,Chat,pqr@abc.com,Technician,Updated service quote available,"As requested, please find the updated service quote. Let us know if it fits your requirements.",Technician provided updated service quote.,CREATE_CHANGE_ORDER,2025-04-24,System
7,SalesPitch,System,System,"New Senior Living Facility Launch in Columbus, OH","We're excited to inform you about a new senior living facility under construction in Columbus, OH. Since you've purchased a shower unit from us, we think you'll love our new range of anti-slip matss that complement your existing choice. Take advantage of our early access deals!","New senior facility in Columbus, OH; recommended anti-slip mats for existing shower unit buyer.",SALES CALL,2025-04-16,System
8,Email,admin@maintain.com,Customer,Follow-up on service request,Even suffer several agreement. Cold bring head day. Law surface sit others back. Institution this pretty test work fall follow.,Followed up via email about service.,ACTION_SEND_EMAIL,2025-04-15,System
9,Chat,ghi@abc.com,Technician,Updated ETA for service,We’ve updated the estimated time of arrival for your scheduled service visit. Thank you for your patience.,Technician provided updated ETA.,GET_UPDATED_ETA,2025-05-05,System
10,Call,8901234567,Technician,Updated ETA for service,We’ve updated the estimated time of arrival for your scheduled service visit. Thank you for your patience.,Technician provided updated ETA.,GET_UPDATED_ETA,2025-05-09,System


In [0]:
input_sqldf = _sqldf
print(input_sqldf.count())

1000


In [0]:
display(input_sqldf)

EventID,EventType,EventFrom,SenderType,Subject,Content,Summary,ActionSuggested,insrt_tmstmp,insrt_ID
1,Email,pqr@abc.com,Technician,Updated service quote available,"As requested, please find the updated service quote. Let us know if it fits your requirements.",Technician provided updated service quote.,CREATE_CHANGE_ORDER,2025-05-23,System
2,Call,9876543210,Technician,Updated ETA for service,We’ve updated the estimated time of arrival for your scheduled service visit. Thank you for your patience.,Technician provided updated ETA.,GET_UPDATED_ETA,2025-06-06,System
3,SalesPitch,System,System,"New Senior Living Facility Launch in Columbus, OH","We're excited to inform you about a new senior living facility under construction in Columbus, OH. Since you've purchased a AC unit from us, we think you'll love our new range of air purifiers that complement your existing choice. Take advantage of our early access deals!","New senior facility in Columbus, OH; recommended air purifier for existing AC unit buyer.",SALES CALL,2025-06-01,System
4,SalesPitch,System,System,"New Senior Living Facility Launch in Orlando, FL","We're excited to inform you about a new senior living facility under construction in Orlando, FL. Since you've purchased a AC unit from us, we think you'll love our new range of air purifiers that complement your existing choice. Take advantage of our early access deals!","New senior facility in Orlando, FL; recommended air purifier for existing AC unit buyer.",SALES CALL,2025-05-15,System
5,Email,abc@xyz.com,Customer,Billing discrepancy,I noticed an unexpected charge on my bill this month. Can someone explain?,Customer inquired about a billing issue.,ESCALATE_TO_BILLING_TEAM,2025-05-18,System
6,Chat,pqr@abc.com,Technician,Updated service quote available,"As requested, please find the updated service quote. Let us know if it fits your requirements.",Technician provided updated service quote.,CREATE_CHANGE_ORDER,2025-04-24,System
7,SalesPitch,System,System,"New Senior Living Facility Launch in Columbus, OH","We're excited to inform you about a new senior living facility under construction in Columbus, OH. Since you've purchased a shower unit from us, we think you'll love our new range of anti-slip matss that complement your existing choice. Take advantage of our early access deals!","New senior facility in Columbus, OH; recommended anti-slip mats for existing shower unit buyer.",SALES CALL,2025-04-16,System
8,Email,admin@maintain.com,Customer,Follow-up on service request,Even suffer several agreement. Cold bring head day. Law surface sit others back. Institution this pretty test work fall follow.,Followed up via email about service.,ACTION_SEND_EMAIL,2025-04-15,System
9,Chat,ghi@abc.com,Technician,Updated ETA for service,We’ve updated the estimated time of arrival for your scheduled service visit. Thank you for your patience.,Technician provided updated ETA.,GET_UPDATED_ETA,2025-05-05,System
10,Call,8901234567,Technician,Updated ETA for service,We’ve updated the estimated time of arrival for your scheduled service visit. Thank you for your patience.,Technician provided updated ETA.,GET_UPDATED_ETA,2025-05-09,System


In [0]:
import pandas as pd

# Create a DataFrame with a single column 'output'
output_df = pd.DataFrame(columns=['EventID', 'Event_Identified',
'Summary','Reasoning','ActionTaken','Inputs'])

for idx, row in enumerate(_sqldf.collect()):
    if idx >= 5: ##TODO Remove this cap
        break
    event_id = row.EventID
    output1 = cs_agent.invoke_agent(row.Content, default_feedback_source)

    output_actions = output1["actions"]
    print(output1)
    if len(output_actions) == 0:
        continue
    action_taken = output_actions[0]["action"]
    action_identified = output_actions[0]["event_identified"]
    reason = output_actions[0]["metadata"].get("purpose", "")
    inputs = output_actions[0]["metadata"].get("params", "")

    output = pd.DataFrame([[event_id, action_identified, '1', reason, action_taken, inputs]], columns=['EventID', 'Event_Identified', 'Summary', 'Reasoning', 'ActionTaken', 'Inputs'])
    
    output_df = pd.concat([output_df,output])

  TOOL CALL: action_close_ticket: Received an updated service quote for review, but no further action required from the assistant at this moment.
{'actions': [{'action': 'CLOSE_TICKET', 'event_identified': 'Other', 'metadata': {'reason': 'Received an updated service quote for review, but no further action required from the assistant at this moment.'}}], 'data_state': {'get_email_activity': {'subject': 'Drive way paving', 'content': 'We have a driveway that needs to be paved. Can I get someone out'}}}
{'actions': [], 'data_state': {'get_email_activity': {'subject': 'Drive way paving', 'content': 'We have a driveway that needs to be paved. Can I get someone out'}}}
{'actions': [{'action': 'ACTION_NOTHING', 'event_identified': 'UNSUPPORTED_EVENT', 'metadata': {'reason': 'The email is promotional and does not require immediate action. It should be handled by the customer service or marketing team for further engagement.'}}], 'data_state': {'get_email_activity': {'subject': 'Drive way pavin

In [0]:
output_df

,EventID,Event_Identified,Summary,Reasoning,ActionTaken,Inputs
0,1,Other,1,,CLOSE_TICKET,
0,3,UNSUPPORTED_EVENT,1,,ACTION_NOTHING,
0,5,Unexpected Charge Inquiry,1,,ACTION_ESCALATE_TICKET,


In [0]:
# Retrieve the most recent run_id from agent_output table
most_recent_run_id = spark.sql("SELECT MAX(run_id) AS max_run_id FROM agent_output").collect()[0]['max_run_id']
display(most_recent_run_id)

output_spark_df = spark.createDataFrame(output_df)
from pyspark.sql.functions import lit

output_spark_df = output_spark_df.withColumn("RunID", lit(most_recent_run_id + 1))
cols = ["RunID"] + [col for col in output_spark_df.columns if col != "RunID"]
output_spark_df = output_spark_df.select(*cols)

output_spark_df = output_spark_df.withColumn("EventID", output_spark_df["EventID"].cast("int"))
display(output_spark_df)

output_spark_df.write.option("mergeSchema", "true").mode("append").saveAsTable("hackathon.default.agent_output")

2

RunID,EventID,Event_Identified,Summary,Reasoning,ActionTaken,Inputs
3,1,Other,1,,CLOSE_TICKET,
3,3,UNSUPPORTED_EVENT,1,,ACTION_NOTHING,
3,5,Unexpected Charge Inquiry,1,,ACTION_ESCALATE_TICKET,


In [0]:
%sql
Select * from agent_output

run_id,EventID,Event_Identified,Summary,Reasoning,ActionTaken,Inputs
2,1,1,1,1,1,1
2,1,1,1,1,1,1
1,1,1,1,1,1,1
1,1,1,1,1,1,1
